In [4]:
exec(open("cleanup.py").read())

In [5]:
import json
from collections import defaultdict

def get_response(fname):
    with open(fname, 'r') as f:
        data = f.read()

    response = json.loads(data)

    surv_resp = defaultdict(dict)
    pass_resp = defaultdict(str)
    time_resp = defaultdict(str)

    for k, v in response.items():
        if 'response' in k:
            temp = k.split("_")
            pid = temp[0].strip('p')
            v = json.loads(v)
            surv_resp[pid].update(v)
        elif 'time' in k:
            temp = k.split("_")
            pid = temp[0]
            time_resp[pid] = float(v)
        elif 'password' in k:
            pass_resp[k] = v

    return (pass_resp, time_resp, surv_resp)

In [6]:
page_map = {
    '0': 'startstudy',
    '1': 'fpass',
    '2': 'fcog',
    '3': 'fpasssurvey',
    '4': 'fpassrecall',
    '5': 'pagegif',
    '6': 'desinint',
    '7': 'quest',
    '8': 'csquest',
    '9': 'spass',
    '10': 'scog',
    '11': 'spasssurvey',
    '12': 'spassrecall',
    '13': 'demo',
    '14': 'thanks',
}

In [7]:
import os
fpath = './responses'
password_response = []
time_response = []
survey_response = []
for i, fname in enumerate(os.listdir(fpath)):
    pss, tim, sur = get_response(os.path.join(fpath, fname))
    password_response.append(pss)
    time_response.append(tim)
    survey_response.append(sur)

In [8]:
import pandas as pd
pd.DataFrame(password_response)

,password1,password2_recall2,password2,password2_recall1,password1_recall1,password2_recall3,password1_recall3,password1_recall2
0,as,123,231,132,asa,312,NaN,NaN
1,21,aw,as,aw,34,NaN,NaN,NaN
2,ddfsfds,NaN,asd,NaN,asda,NaN,ddsda,dasdas


In [9]:
page_time = pd.DataFrame(time_response)
page_time

,10,0,6,9,3,2,12,5,8,7,11,4,1
0,3.851,8.220,18.979,15.644,11.823,2.212,9.328,11.319,29.334,28.927,13.077,9.787,30.243
1,2.694,5.809,17.147,38.820,15.487,3.239,12.289,3.054,NaN,19.531,NaN,7.095,7.116
2,6.824,3.295,4.320,5.639,10.457,3.496,3.141,2.276,NaN,19.845,NaN,6.747,12.029


In [10]:
# top 5 pages with higher time
top5 = pd.DataFrame(columns=['page', 'time'])
for i, data in page_time.mean().sort_values(ascending=False).head(5).items():
    top5 = top5.append(pd.DataFrame([[page_map.get(i), data]], columns=top5.columns), ignore_index=True)

top5

,page,time
0,csquest,29.334000
1,quest,22.767667
2,spass,20.034333
3,fpass,16.462667
4,desinint,13.482000


In [11]:
pd.DataFrame(survey_response)

,7,8,13,11,3
0,"{'1-19': '1', '1-20': '2', '1-21': '3', '1-22'...","{'3-1': '4', '4-1': '4', '5-1': ['Social Media...","{'1-1': 'Woman', '2-1': '18-24 years old', '3-...","{'1-1': '7', '2-1': '7', '2-2': '7', '2-3': '7...","{'1-1': '4', '2-1': '5', '2-2': '5', '2-3': '5..."
1,"{'1-19': '5', '1-20': '5', '1-21': '5', '1-22'...","{'1-1': '7', '1-2': '7', '1-3': '7', '1-4': '7...","{'1-1': 'Transgender/Trans woman', '2-1': '35-...","{'1-1': '7', '2-1': '7', '2-2': '7', '2-3': '7...","{'1-1': '3', '2-1': '4', '2-2': '4', '2-3': '4..."
2,"{'1-19': '3', '1-20': '3', '1-21': '4', '1-22'...","{'1-1': '3', '1-2': '4', '1-3': '4', '1-4': '5...","{'1-1': 'Man', '2-1': '40-44 years old', '3-1'...","{'1-1': '4', '2-1': '4', '2-2': '5', '2-3': '5...","{'1-1': '3', '2-1': '4', '2-2': '5', '2-3': '6..."
